Section 1: Data Loading and Preprocessing


In [1]:
from PIL import Image
import numpy as np
import os
from keras.utils import to_categorical

# Define image size
IMG_SIZE = 224

# Define categories and their labels
CATEGORIES = {"daisy": 0, "dandelion": 1, "roses": 2, "sunflowers": 3, "tulips": 4}

# Initialize X and y arrays
X = []
y = []
label_counts = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}  # keep track of the number of images added for each category
categories = list(CATEGORIES.keys())  # get a list of category names
while True:
    stop = True
    for category in categories:
        if label_counts[CATEGORIES[category]] < len(os.listdir(os.path.join("small_flower_dataset", category))):
            # Load the next image and add it to X and y
            image_path = os.path.join("small_flower_dataset", category, os.listdir(os.path.join("small_flower_dataset", category))[label_counts[CATEGORIES[category]]])
            image = Image.open(image_path).resize((IMG_SIZE, IMG_SIZE))
            image_array = np.array(image)
            X.append(image_array)
            y.append(CATEGORIES[category])
            label_counts[CATEGORIES[category]] += 1
            stop = False
    if stop:
        break

# Convert X and y to numpy arrays
X = np.array(X)
y = np.array(y)

# Split the data into train, Test and validation sets
split1 = int(len(X) * 0.6)  # 60% train
split2 = int(len(X) * 0.8)  # 20% validation

# Create the train set
X_train, y_train = X[:split1], y[:split1]

# Create the validation set
X_val, y_val = X[split1:split2], y[split1:split2]

# Create the test set
X_test, y_test = X[split2:], y[split2:]

num_classes = len(CATEGORIES)
y_train = to_categorical(y_train, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)


Section 2: Model Training

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from keras.utils import to_categorical

# Load the MobileNetV2 model pre-trained on ImageNet
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze all the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add your own output layer with 5 neurons and a softmax activation function
x = Flatten()(base_model.output)
x = Dense(5, activation='softmax')(x)

# Create a new model that includes both the pre-trained base model and your custom output layer
model = Model(inputs=base_model.input, outputs=x)

# Define the optimizer
optimizer = SGD(learning_rate=0.001, momentum=0.9, nesterov=False)

# Compile the model with an appropriate optimizer, loss function, and metrics
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


# Train the model on your own dataset
model.fit(X_train, y_train, epochs=25, validation_data=(X_val, y_val))


Epoch 1/25
19/19 [==============================] - 27s 1s/step - loss: 16.8603 - accuracy: 0.2417 - val_loss: 17.3193 - val_accuracy: 0.3450
Epoch 2/25
19/19 [==============================] - 20s 1s/step - loss: 8.8404 - accuracy: 0.5483 - val_loss: 10.3336 - val_accuracy: 0.4300
Epoch 3/25
19/19 [==============================] - 20s 1s/step - loss: 3.1718 - accuracy: 0.7250 - val_loss: 7.1257 - val_accuracy: 0.4950
Epoch 4/25
19/19 [==============================] - 20s 1s/step - loss: 1.7044 - accuracy: 0.8150 - val_loss: 7.4732 - val_accuracy: 0.4950
Epoch 5/25
19/19 [==============================] - 20s 1s/step - loss: 1.2947 - accuracy: 0.8600 - val_loss: 6.2685 - val_accuracy: 0.5350
Epoch 6/25
19/19 [==============================] - 20s 1s/step - loss: 0.2956 - accuracy: 0.9433 - val_loss: 5.5694 - val_accuracy: 0.5100
Epoch 7/25
19/19 [==============================] - 21s 1s/step - loss: 0.1251 - accuracy: 0.9700 - val_loss: 5.8931 - val_accuracy: 0.5150
Epoch 8/25
19/19 

Section 3: Prediction and Visualization

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Predict the first 10 results in X_test
y_pred = model.predict(X_test)
y_pred_labels = [list(CATEGORIES.keys())[list(CATEGORIES.values()).index(np.argmax(p))] for p in y_pred]

# Convert the one-hot encoded true labels to labels
y_true_labels = list(y_test)
true_labels = []
for label in y_true_labels:
    true_labels.append(list(CATEGORIES.keys())[list(CATEGORIES.values()).index(np.argmax(label))])

fig, axes = plt.subplots(nrows=5, ncols=10, figsize=(20,10))

for i, ax in enumerate(axes.flat):
    ax.imshow(X_test[i])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f"True: {true_labels[i]}\nPredicted: {y_pred_labels[i]}")

plt.tight_layout()
plt.show()


NameError: name 'model' is not defined